In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
import json
import pandas as pd

In [2]:
spark = spark = SparkSession.builder \
    .appName("pyspark_test") \
    .getOrCreate()

25/03/01 21:34:36 WARN Utils: Your hostname, Harrys-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.0.0.118 instead (on interface en0)
25/03/01 21:34:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/01 21:34:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 53267)
Traceback (most recent call last):
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/socketserver.py", line 747, in __init__
    self.handle()
  File "/Users/harrymierz/Desktop/Projects/Valorant/.

In [3]:
def load_file(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

file_path = '../data/Team_Liquid_vs_Sentinels.json'

data = load_file(file_path)

### get match event data

In [4]:
def get_match_info():
    match_name = list(data.keys())[0]

    team1, team2 = match_name.split('_vs_')
    team1_name = team1.replace('_', ' ')
    team2_name = team2.replace('_', ' ')


    event_stats = data[match_name][0]
    event_stats

    return match_name, team1_name, team2_name

match_name, team1_name, team2_name = get_match_info()

In [5]:
map_list = list(data[match_name][1].keys())

### get team map data

In [6]:
# Function that gets team stats for each map in the match
def get_team_map_data(match_name, map_list, data):

    map_team_stats_df = pd.DataFrame(columns = ['map_name', 'team_name', 'win' ,'final_score', 'defense_score' ,'attack_score', 'overtime_score'])

    for i in map_list:

        team1_stats = data[match_name][1][i]['Team Stats: '][team1_name]
        team2_stats = data[match_name][1][i]['Team Stats: '][team2_name]

        team1_score = int(team1_stats['final'])
        team2_score = int(team2_stats['final'])

        winner = team1_name if int(team1_score) > int(team2_score) else team2_name

        if winner == team1_name:
            team1_win = True
            team2_win = False
        else:
            team1_win = False
            team2_win = True


        team1_dict = {'map_name' : [i], 'team_name' : [team1_name], 'win' : [team1_win], 'final_score' : [team1_stats['final']], 'defense_score' : [team1_stats['CT']], 'attack_score' : [team1_stats['T']], 'overtime_score' : [team1_stats['OT']]}
        team2_dict = {'map_name' : [i], 'team_name' : [team2_name], 'win' : [team2_win], 'final_score' : [team2_stats['final']], 'defense_score' : [team2_stats['CT']], 'attack_score' : [team2_stats['T']], 'overtime_score' : [team2_stats['OT']]}

        team1_df = pd.DataFrame(team1_dict)
        team2_df = pd.DataFrame(team2_dict)


        map_team_stats_df = pd.concat([map_team_stats_df, team1_df, team2_df], ignore_index=True)


    return spark.createDataFrame(map_team_stats_df), map_team_stats_df

map_team_stats_df_spark, map_team_stats_df  = get_team_map_data(match_name, map_list, data)

print('Map Team Data')
map_team_stats_df_spark.show()

Map Team Data


+-----+-----------+-----+-----------+-------------+------------+--------+--------------+
|  map|  team_name|  win|final_score|defense_score|attack_score|ot_score|overtime_score|
+-----+-----------+-----+-----------+-------------+------------+--------+--------------+
|Lotus|Team Liquid|false|          7|            2|           5|     NaN|           0.0|
|Lotus|  Sentinels| true|         13|            7|           6|     NaN|           0.0|
|Abyss|Team Liquid| true|         16|            7|           5|     NaN|             4|
|Abyss|  Sentinels|false|         14|            7|           5|     NaN|             2|
| Bind|Team Liquid| true|         13|           11|           2|     NaN|             0|
| Bind|  Sentinels|false|          1|            0|           1|     NaN|             0|
+-----+-----------+-----+-----------+-------------+------------+--------+--------------+



In [7]:
def get_team_abbreviation(data, match_name, map_list):

    match_name = list(data.keys())[0]

    team1, team2 = match_name.split('_vs_')
    team1_name = team1.replace('_', ' ')
    team2_name = team2.replace('_', ' ')

    map_name = map_list[0]

    first_player = list(data[match_name][1][map_name]['Player Stats'].keys())[0]
    last_player = list(data[match_name][1][map_name]['Player Stats'].keys())[-1]

    team1_abbr = data[match_name][1][map_name]['Player Stats'][first_player]["Team"]
    
    team2_abbr = data[match_name][1][map_name]['Player Stats'][last_player]["Team"]

    team_abbr_dict = {team1_abbr : team1_name, team2_abbr : team2_name}

    return team_abbr_dict

team_name_abbr_dict = get_team_abbreviation(data, match_name, map_list)

print(team_name_abbr_dict)


{'TL': 'Team Liquid', 'SEN': 'Sentinels'}


In [8]:
def get_winner(match_name, map_list, data):

    map_win_list = []

    for i in map_list:

        team1_stats = data[match_name][1][i]['Team Stats: '][team1_name]
        team2_stats = data[match_name][1][i]['Team Stats: '][team2_name]

        team1_score = int(team1_stats['final'])
        team2_score = int(team2_stats['final'])
    

        map_win_list.append(team1_name if int(team1_score) > int(team2_score) else team2_name)

    return max(set(map_win_list), key=map_win_list.count)

winner = get_winner(match_name, map_list, data)
print(f'Winner: {winner}')

Winner: Team Liquid


In [9]:
# Function that gets event and date stats for a match
def get_match_event_stats(team1_name, team2_name, winner, data, map_list):

    event_stats_df_pd = pd.DataFrame(data[list(data.keys())[0]][0], index=[0])

    event_stats_df_pd = event_stats_df_pd.rename(columns={'Event' : 'event', 'Date' : 'match_date', 'Stage' : 'stage', 'Round' : 'round'})

    event_stats_df_pd['winner'] = winner
    event_stats_df_pd['team_one_name'] = team1_name
    event_stats_df_pd['team_two_name'] = team2_name
    try:
        event_stats_df_pd['map_one'] = map_list[0]
    except IndexError:
         event_stats_df_pd['map_one'] = None
    try:
        event_stats_df_pd['map_two'] = map_list[1] 
    except IndexError:
        event_stats_df_pd['map_two'] = None
    try: 
        event_stats_df_pd['map_three'] = map_list[2] 
    except IndexError:
        event_stats_df_pd['map_three'] = None
    try: 
        event_stats_df_pd['map_four'] = map_list[3] 
    except IndexError:
        event_stats_df_pd['map_four'] = None
    try:
        event_stats_df_pd['map_five'] = map_list[4]
    except IndexError:
        event_stats_df_pd['map_five'] = None

    schema = StructType([
        StructField("event", StringType(), True),
        StructField("match_date", StringType(), True),
        StructField("stage", StringType(), True),
        StructField("round", StringType(), True),
        StructField("winner", StringType(), True),
        StructField("team_one_name", StringType(), True),
        StructField("team_two_name", StringType(), True),
        StructField("map_one", StringType(), True),
        StructField("map_two", StringType(), True),
        StructField("map_three", StringType(), True),
        StructField("map_four", StringType(), True),
        StructField("map_five", StringType(), True),
    ])

    event_stats_df_spark = spark.createDataFrame(event_stats_df_pd, schema=schema)

    return event_stats_df_spark

match_event_info_df = get_match_event_stats(team1_name, team2_name, winner, data, map_list)

print('Match Event Data')
match_event_info_df.show()

Match Event Data
+--------------------+----------+-----------+-------------+-----------+-------------+-------------+-------+-------+---------+--------+--------+
|               event|match_date|      stage|        round|     winner|team_one_name|team_two_name|map_one|map_two|map_three|map_four|map_five|
+--------------------+----------+-----------+-------------+-----------+-------------+-------------+-------+-------+---------+--------+--------+
|Champions Tour 20...|2025-02-23|Swiss Stage|Round 2 (0-1)|Team Liquid|  Team Liquid|    Sentinels|  Lotus|  Abyss|     Bind|    NULL|    NULL|
+--------------------+----------+-----------+-------------+-----------+-------------+-------------+-------+-------+---------+--------+--------+



### get map player data

In [10]:
# Function that gets player stats for each map in the match
def get_map_player_data(map_list, team_name_abbr_dict, data, match_name):

    map_player_stats_df_pd = pd.DataFrame(columns=['map', 'player_name', 'agent', 'team_name', 'acs_overall','acs_attack', 'acs_defense', 'kills_overall', 'kills_attack', 'kills_defense',  'deaths_overall', 'deaths_attack', 'deaths_defense', 'assists_overall', 'assists_attack', 'assists_defense', 'kast_overall', 'kast_attack', 'kast_defense', 'adr_overall', 'adr_attack', 'adr_defense', 'headshot_percentage_overall', 'headshot_percentage_attack', 'headshot_percentage_defense', 'first_kills_overall', 'first_kills_attack', 'first_kills_defense', 'first_deaths_overall', 'first_deaths_attack', 'first_deaths_defense'])

    map_player_stats_df_pd['kast_overall'] = map_player_stats_df_pd['kast_overall'].str.replace('%', '').astype(float)
    map_player_stats_df_pd['kast_defense'] = map_player_stats_df_pd['kast_defense'].str.replace('%', '').astype(float)
    map_player_stats_df_pd['kast_attack'] = map_player_stats_df_pd['kast_attack'].str.replace('%', '').astype(float)
    map_player_stats_df_pd['headshot_percentage_overall'] = map_player_stats_df_pd['headshot_percentage_overall'].str.replace('%', '').astype(float)
    map_player_stats_df_pd['headshot_percentage_defense'] = map_player_stats_df_pd['headshot_percentage_defense'].str.replace('%', '').astype(float)
    map_player_stats_df_pd['headshot_percentage_attack'] = map_player_stats_df_pd['headshot_percentage_attack'].str.replace('%', '').astype(float)



    for i in map_list:
        map_name = i

        map_player_stats = data[match_name][1][i]['Player Stats']
    
        for player in map_player_stats:
            player_name = player
            player_stats = map_player_stats[player]

            temp_df3 = pd.DataFrame({'map' : map_name,'player_name': player_name, 'agent': player_stats['Agent'], 'team_name' : team_name_abbr_dict[player_stats['Team']], 'acs_overall': player_stats['ACS']['All'], 'acs_attack':player_stats['ACS']['T'], 'acs_defense': player_stats['ACS']['CT'], 'kills_overall' : player_stats['Elims']['All'], 'kills_attack': player_stats['Elims']['T'], 'kills_defense': player_stats['Elims']['CT'], 'deaths_overall' : player_stats['Deaths']['All'], 'deaths_attack': player_stats['Deaths']['T'], 'deaths_defense': player_stats['Deaths']['CT'], 'assists_overall' : player_stats['Assists']['All'], 'assists_attack': player_stats['Assists']['T'], 'assists_defense': player_stats['Assists']['CT'], 'kast_overall' :player_stats['KAST']['All'], 'kast_attack': player_stats['KAST']['T'], 'kast_defense': player_stats['KAST']['CT'], 'adr_overall' : player_stats['ADR']['All'], 'adr_attack': player_stats['ADR']['T'], 'adr_defense': player_stats['ADR']['CT'], 'headshot_percentage_overall' : player_stats['HS_percentage']['All'], 'headshot_percentage_attack': player_stats['HS_percentage']['T'], 'headshot_percentage_defense': player_stats['HS_percentage']['CT'], 'first_kills_overall' : player_stats['First Kills']['All'], 'first_kills_attack': player_stats['First Kills']['T'], 'first_kills_defense': player_stats['First Kills']['CT'], 'first_deaths_overall' : player_stats['First Deaths']['All'], 'first_deaths_attack': player_stats['First Deaths']['T'], 'first_deaths_defense': player_stats['First Deaths']['CT']}, index=[0])

            map_player_stats_df_pd = pd.concat([map_player_stats_df_pd, temp_df3], ignore_index=True)

    return map_player_stats_df_pd

map_player_stats_df__spark = spark.createDataFrame(get_map_player_data(map_list, team_name_abbr_dict, data, match_name))

print('Map Player Data')
map_player_stats_df__spark.show(5)

Map Player Data
+-----+-----------+------+-----------+-----------+----------+-----------+-------------+------------+-------------+--------------+-------------+--------------+---------------+--------------+---------------+------------+-----------+------------+-----------+----------+-----------+---------------------------+--------------------------+---------------------------+-------------------+------------------+-------------------+--------------------+-------------------+--------------------+
|  map|player_name| agent|  team_name|acs_overall|acs_attack|acs_defense|kills_overall|kills_attack|kills_defense|deaths_overall|deaths_attack|deaths_defense|assists_overall|assists_attack|assists_defense|kast_overall|kast_attack|kast_defense|adr_overall|adr_attack|adr_defense|headshot_percentage_overall|headshot_percentage_attack|headshot_percentage_defense|first_kills_overall|first_kills_attack|first_kills_defense|first_deaths_overall|first_deaths_attack|first_deaths_defense|
+-----+-----------

25/03/01 21:34:40 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


### get match aggregate data

### team data

In [11]:
# Function that gets team stats for each map in the match
def get_agg_team_data(match_name, map_list, data):

    map_team_df_spark, map_team_df_pd = get_team_map_data(match_name, map_list, data)

    map_team_df_pd = map_team_df_pd[['team_name', 'final_score', 'defense_score', 'attack_score', 'overtime_score']]

    map_team_df_pd = map_team_df_pd.astype({'final_score' : int, 'defense_score' : int, 'attack_score' : int, 'overtime_score' : int})

    match_team_sums = map_team_df_pd.groupby('team_name').sum()

    match_team_sums.reset_index(inplace=True)

    return spark.createDataFrame(match_team_sums), match_team_sums

team_match_agg_stats_spark, team_match_agg_stats_df  = get_agg_team_data(match_name, map_list, data)

print('Map Team Data')
team_match_agg_stats_spark.show()

Map Team Data
+-----------+-----------+-------------+------------+--------------+
|  team_name|final_score|defense_score|attack_score|overtime_score|
+-----------+-----------+-------------+------------+--------------+
|  Sentinels|         28|           14|          12|             2|
|Team Liquid|         36|           20|          12|             4|
+-----------+-----------+-------------+------------+--------------+



### Player Data

In [12]:
# Function to get aggregate player data for the entire match over all maps
def get_agg_player_data(map_list, team_name_abbr_dict, data, match_name):
    map_player_stats_df_pd = get_map_player_data(map_list, team_name_abbr_dict, data, match_name)

    map_player_stats_df_pd_trim = map_player_stats_df_pd.copy()

    map_player_stats_df_pd_trim['kast_overall'] = map_player_stats_df_pd_trim['kast_overall'].str.replace('%', '').astype(float)
    map_player_stats_df_pd_trim['kast_defense'] = map_player_stats_df_pd_trim['kast_defense'].str.replace('%', '').astype(float)
    map_player_stats_df_pd_trim['kast_attack'] = map_player_stats_df_pd_trim['kast_attack'].str.replace('%', '').astype(float)
    map_player_stats_df_pd_trim['headshot_percentage_overall'] = map_player_stats_df_pd_trim['headshot_percentage_overall'].str.replace('%', '').astype(float)
    map_player_stats_df_pd_trim['headshot_percentage_defense'] = map_player_stats_df_pd_trim['headshot_percentage_defense'].str.replace('%', '').astype(float)
    map_player_stats_df_pd_trim['headshot_percentage_attack'] = map_player_stats_df_pd_trim['headshot_percentage_attack'].str.replace('%', '').astype(float)

    map_player_stats_df_pd_trim = map_player_stats_df_pd_trim.astype({'acs_overall': 'int', 'acs_attack': 'int', 'acs_defense':  'int', 'kills_overall' : 'int', 'kills_attack': 'int', 'kills_defense': 'int', 'deaths_overall' : 'int', 'deaths_attack': 'int', 'deaths_defense': 'int', 'assists_overall' : 'int', 'assists_attack': 'int', 'assists_defense': 'int', 'kast_overall' : 'float', 'kast_attack':  'float', 'kast_defense':  'float', 'adr_overall' :  'int', 'adr_attack':  'int', 'adr_defense':  'int', 'headshot_percentage_overall' :  'float', 'headshot_percentage_attack':  'float', 'headshot_percentage_defense':  'float', 'first_kills_overall' : 'int', 'first_kills_attack': 'int', 'first_kills_defense': 'int', 'first_deaths_overall' : 'int', 'first_deaths_attack': 'int', 'first_deaths_defense': 'int'})

    agg_player_stats_df_pd = map_player_stats_df_pd_trim.groupby('player_name').agg({'acs_overall': 'mean', 'acs_attack': 'mean', 'acs_defense':  'mean', 'kills_overall' : 'sum', 'kills_attack': 'sum', 'kills_defense': 'sum', 'deaths_overall' : 'sum', 'deaths_attack': 'sum', 'deaths_defense': 'sum', 'assists_overall' : 'sum', 'assists_attack': 'sum', 'assists_defense': 'sum', 'kast_overall' : 'mean', 'kast_attack':  'mean', 'kast_defense':  'mean', 'adr_overall' :  'mean', 'adr_attack':  'mean', 'adr_defense':  'mean', 'headshot_percentage_overall' :  'mean', 'headshot_percentage_attack':  'mean', 'headshot_percentage_defense':  'mean', 'first_kills_overall' : 'sum', 'first_kills_attack': 'sum', 'first_kills_defense': 'sum', 'first_deaths_overall' : 'sum', 'first_deaths_attack': 'sum', 'first_deaths_defense': 'sum'}).round(2)

    agg_player_stats_df_pd.reset_index(inplace=True)

    return agg_player_stats_df_pd

agg_player_stats_df_spark = spark.createDataFrame(get_agg_player_data(map_list, team_name_abbr_dict, data, match_name))

print('Aggregate Player Data')
agg_player_stats_df_spark.show()

Aggregate Player Data
+-----------+-----------+----------+-----------+-------------+------------+-------------+--------------+-------------+--------------+---------------+--------------+---------------+------------+-----------+------------+-----------+----------+-----------+---------------------------+--------------------------+---------------------------+-------------------+------------------+-------------------+--------------------+-------------------+--------------------+
|player_name|acs_overall|acs_attack|acs_defense|kills_overall|kills_attack|kills_defense|deaths_overall|deaths_attack|deaths_defense|assists_overall|assists_attack|assists_defense|kast_overall|kast_attack|kast_defense|adr_overall|adr_attack|adr_defense|headshot_percentage_overall|headshot_percentage_attack|headshot_percentage_defense|first_kills_overall|first_kills_attack|first_kills_defense|first_deaths_overall|first_deaths_attack|first_deaths_defense|
+-----------+-----------+----------+-----------+-------------+

### Get Team Info Data

In [13]:
#Function to get team info data

def get_team_info(match_name, map_list,data,team1_name, team2_name):

    team1_abbr, team2_abbr = get_team_abbreviation(data, match_name, map_list)

    team_info_dict = {
        'TeamName' : [team1_name, team2_name],
        'TeamAbbreviation' : [team1_abbr, team2_abbr]
    }

    team_info_df = pd.DataFrame(team_info_dict)



    return spark.createDataFrame(team_info_df), team_info_df

team_info_df_spark, team_info_df_pd = get_team_info(match_name, map_list, data, team1_name, team2_name)
team_info_df_spark.show()

+-----------+----------------+
|   TeamName|TeamAbbreviation|
+-----------+----------------+
|Team Liquid|              TL|
|  Sentinels|             SEN|
+-----------+----------------+



### Get Player Info Data

In [14]:
# Function to get player info data
def get_player_info(map_list, team_name_abbr_dict, data, match_name):
    map_player_stats_df_pd = get_map_player_data(map_list, team_name_abbr_dict, data, match_name)

    player_name_list = map_player_stats_df_pd['player_name'].unique()

    team1_name = map_player_stats_df_pd['team_name'].unique()[0]
    team2_name = map_player_stats_df_pd['team_name'].unique()[1]



    player_info_df = pd.DataFrame({'player_name' : player_name_list, 'team_name' : [i for i in range(10)]})

    player_info_df.loc[0:4, 'team_name'] = team1_name
    player_info_df.loc[5:9, 'team_name'] = team2_name

    return player_info_df

player_info_df_spark = spark.createDataFrame(get_player_info(map_list, team_name_abbr_dict, data, match_name))
player_info_df_spark.show()

+-----------+-----------+
|player_name|  team_name|
+-----------+-----------+
|       nAts|Team Liquid|
|      Keiko|Team Liquid|
|    paTiTek|Team Liquid|
|      kamyk|Team Liquid|
|       kamo|Team Liquid|
|       bang|  Sentinels|
|     zekken|  Sentinels|
|    N4RRATE|  Sentinels|
|     johnqt|  Sentinels|
|    Zellsis|  Sentinels|
+-----------+-----------+



/var/folders/9q/9pfj8gwj3slddlkchn462w_80000gn/T/ipykernel_17918/4181393774.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Team Liquid' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  player_info_df.loc[0:4, 'team_name'] = team1_name


### Get Map Info Data

In [15]:
# Function to get map info data
def get_map_info(map_list):

    map_df = pd.DataFrame({'MapName' : map_list})

    return spark.createDataFrame(map_df), map_df

map_df_spark, map_df_pd = get_map_info(map_list)
map_df_spark.show()

+-------+
|MapName|
+-------+
|  Lotus|
|  Abyss|
|   Bind|
+-------+



25/03/02 01:51:05 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 978925 ms exceeds timeout 120000 ms
25/03/02 01:51:05 WARN SparkContext: Killing executors is not supported by current scheduler.
25/03/02 01:51:13 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$